# Converting LiDAR data into voxels for use in Minetest worlds

## Credit
The following notebook was inspired and adapted from [James Wooton](https://gist.github.com/quantumjim/2d9ea27d0e0428a537b953ac04d3721b).

[Prof. Paul Pickell](https://github.com/pauldpickell) (University of British Columbia) added (October 28, 2021) additional instructions and functionality to change the scale and fill holes in the ground using inverse distance weighting interpolation. Support for additional map layers beyond LiDAR data coming soon.

Licensed under [GNU General Public License v3.0](https://www.gnu.org/licenses/gpl-3.0.en.html).

## Before you get started
- Ensure that you have the packages the following packages installed for python: `pylas`, `numpy`, and `pandas`. You can call `pip install pylas` within your Python IDE, for example.
- [Download Minetest](https://www.minetest.net/downloads/). To install, simply extract the contents somewhere that you have read/write access to on your computer (e.g., your user directory on Windows `C:\Users\YourUsername\minetest\`). Navigate to the `bin` folder, find the `minetest.exe` executable, right-click it and create a shortcut on your desktop or taskbar, then double-click the shortcut to run Minetest.
- [Download the csv2terrain mod](https://github.com/quantumjim/csv2terrain/blob/master/README.md). To install, simply extract the contents into your `mods` folder within your Minetest directory. For example, `C:\Users\YourUsername\minetest\mods\`. Ensure that the folder is renamed `csv2terrain`. Run Minetest and create a new world with any options then press the `Select Mods` button and you should see `csv2terrain` listed there if the installation was done correctly.

In [ ]:
import pylas
import numpy as np
import random
import pandas as pd
import math
import datetime
import scipy
from skimage.feature import peak_local_max
from scipy import ndimage
from pyproj import Proj
import urllib.request
import xml.etree.ElementTree as ET

Put this notebook in the same folder as your LiDAR data or otherwise indicate the pathname and filename for your data. This file can be in either `.las` or `.laz` format. It is assumed that the ground has been classified in this LiDAR point cloud.

In [ ]:
file = 'MKRF_lidar.las'

## Map centre
Define the centre coordinate of the map. This does not necessarily need to be the centre of the las file, but that is an obvious choice. To find this coordinate, open the las file and calculate the range of the Northings and Eastings, which is usually 1000 or 2000 meters for a typical LiDAR tile. Divide the range by 2 and add that value to the value of the Left extent bound and also to the Bottom extent bound. (Usually LiDAR tiles are square, so you can safely add the same value to the lower bounds of the extent, but you may need to adjust this approach.) 

The Minetest map that you produce is not necessarily going to be the same extent as the LiDAR tile, so you can place the map centre anywhere you want within the las extent. However, you must ensure that this coordinate is at least `size/2` distance from the true edge of the las extent so that you actually build a full map. This coordinate is also used as the starting place for the player.

In [ ]:
# Malcolm Knapp Research Forest
coord = (533500,5464500)

## Map size
Specify the `size` of the output Minetest map. This is the width and length of the map in blocks. Each block corresponds to 1 meter in the point cloud. Thus, this dimension is what will be used to clip a square extent of `size*size` area from the las file centred at `coord`. Increasing this value does not produce smaller voxels, but instead increases the extent of the Minetest map because the size of blocks in Minetest is fixed. Each incremental value of `size` results in a squared number of new positions to be created and potentially many more blocks, so be careful about creating large maps as they can take long to process and may crash Minetest when loaded. You can also try running without the inverse distance weighting interpolation first to see if holes might be a problem in your las. Comment lines 40-56 in the `get_blocks()` function to turn off the interpolation.

In [ ]:
size = 100

## Load and preprocess Open Street Maps data

Now we can retreive the Open Street Maps data using the [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API). Since we may not know exactly what types of features lay within a given point cloud, we will just query the API to return all features that fall within a bounding box defined by our map `size` and point cloud `coord`.

Note that you may need to adjust the projection information below to conform with the projected coordinate system for your las file. Normally, las files are projected into Universal Transverse Mercator (UTM), but you may still need to specify the specific zone for your point cloud location (use the tool below). 

In [ ]:
# use this tool to figure out your UTM zone using the western-most longitude of your area-of-interest
westernLongitudeDecimalDegrees = -120
UTMzone = int((westLongitudeDecimalDegrees+180)/6)
print('UTM Zone is '+str(UTMzone)+'. This value has been passed to the next code chunk.')

In [ ]:
p = Proj(proj='utm',zone=UTMzone,ellps='WGS84')
south = p(coord[0],coord[1]-size/2,inverse=True)[1]
west = p(coord[0]-size/2,coord[1],inverse=True)[0]
north = p(coord[0],coord[1]+size/2,inverse=True)[1]
east = p(coord[0]+size/2,coord[1],inverse=True)[0]
print(south,west,north,east)
print("northings in range of ",int(coord[1]-size/2),int(coord[1]+size/2))
print("eastings in range of ",int(coord[0]-size/2),int(coord[0]+size/2))
urllib.request.urlretrieve('https://overpass-api.de/api/map?bbox='+str(west).strip()+','+str(south).strip()+','+str(east).strip()+','+str(north).strip(), 'map.osm')

Retreive some simple classes from the OSM data and define some block types for those. A full list of OSM map features [can be found here](https://wiki.openstreetmap.org/wiki/Map_features).

## Load and preprocess LiDAR data

`get_points()` applies an `x,y` mask to the point cloud defined by the `size` variable above and then retrieves the `x,y,z` coordinates of all points and rescales the `xy` values to `0 to size` and the `z` values to `0 to max(z)`. Normally, we divide the units of the point cloud by 100 to convert from centimeters to meters, which is approximately what is used by Minetest. You can modify this parameter by choosing any value for `scale`:
- `scale < 1` causes the Minetest world to expand relative to the player (player shrinks)
- `scale = 1` is approximately 1:1 real-world scale
- `scale > 1` causes the Minetest world to shrink relative to the player (player expands)

**Note that if you choose any value other than 1 you will be modifying the scale of the Minetest map (i.e., creating more or fewer voxels relative to the player's perspective).** This parameter does not modify the map `size`. If `scale < 1`, then more voxels are created (point cloud expands *beyond* the map) and any in excess of `size` are dropped and will not be mapped. If `scale > 1`, then fewer voxels are created (point cloud shrinks *within* the map) within the map and some areas of the map `size` may not be used at all. Modifying `scale` should only be used to explore the point cloud data and visualize how different vozel sizes might impact how features might be represented differently.

In [ ]:
scale = 1

In [ ]:
def get_points(x0,y0,file,size,scale):
    # define scale in terms of units (centimeters)
    scale = int(100*scale)
    # load in the LiDAR point cloud
    cloud = pylas.read(file)
    x, y, z = cloud.x.copy(), cloud.y.copy(), cloud.z.copy()
    # create a mask using the size variable, the LiDAR point cloud will be clipped by this mask
    mask = (x>=x0-size/2) & (x<x0+size/2) & (y>=y0-size/2) & (y<y0+size/2)
    # below for creating dataframe to fill missing values
    cm = pd.DataFrame(cloud.points[mask])
    min_x = min(cm.X)
    xx = ((cm.X-min_x)/scale).astype(int)
    min_y = min(cm.Y)
    yy = ((cm.Y-min_y)/scale).astype(int)
    # need to select only returns classified as ground, so that all normalized heights are relative to the terrain surface
    # multipath errors common to urban point clouds may cause the min_z value to be quite large (e.g., -200 m)
    # which then adds 200 m to the terrain and creates a "city in the sky" situation
    # note that negative ground heights are potentially still valid within urban point clouds (e.g., open excavation sites)
    c = cm.raw_classification.copy()
    min_z = min(z[c[c==2]])
    zz = ((cm.Z-min_z)/scale).astype(int)
    xyzc = pd.concat([xx,yy,zz,pd.Series(c)],axis=1)
    xyzc = xyzc.rename({'raw_classification': 'C'}, axis='columns')
    # remove points with NA for any x, y, z, c value
    xyzc = xyzc.dropna(axis=0)
    # below for fast creation of existing locations with z values using dictionaries (there may be holes to fill)
    height = {}
    classification = {}
    for point in cloud.points[mask]:
        xxx = int((point[0]-min_x)/scale)
        yyy = int((point[1]-min_y)/scale)
        height[xxx,yyy] = int((point[2]-min_z)/scale)
        classification[xxx,yyy] = point[5]
    return height,classification,xyzc

## LiDAR block types
Define the block types that you want to associate with the LiDAR classification values. The typical classification scheme is in the following order:
- 0 = unclassified
- 1 = unassigned
- 2 = ground
- 3 = low vegetation
- 4 = medium vegetation
- 5 = high vegetation
- 6 = buildings
- anything over 6 is noise or reserved for another class

[View all block types (nodes) here.](https://wiki.minetest.net/Games/Minetest_Game/Nodes) 

It is best to use solid blocks and avoid blocks that might not stack well like plants and fungi. When you find a block type you want to use, enter the `Itemstring` for the block that corresponds to the classification value you want to associate it with. The block types below offer nice contrast between vegetation types, buildings and the ground. If you want to experiment with different configurations, find a good starting location (i.e., `coord`) with some variety and set the map `size` to a smaller value (e.g., between 20 and 50) so that you can quickly try out different block types. You can change the top layer of the ground by specifying an `Itemstring` for `top_layer_of_ground_block`. You must specify at least the first 3 classes in order to generate the terrain. You can specify more classes, but any excess classes specified that are not classified in the las file will be ignored and you will see a warning message.

**Note that in order to use some block types like slabs, stairs and some items, you need to use and install the `csv2terrain` mod that is distributed with this repository.** Currently, only the default blocks (`default:`), slabs and stairs (`stairs:`), and wool (`wool:`) are supported. Any other block type will crash Minetest (i.e., `doors:`, `flowers:`, `fireflies:`, `tnt:`, `vessels:`, etc). Wool can be a useful block type because it provides a range of colours to work with. To invoke a wool block colour, simply name the colour in the block type (e.g., `cyan`). The following wool colours are available:
- `white`
- `grey`
- `dark_grey`
- `black`
- `blue`
- `cyan`
- `green`
- `dark_green`
- `yellow`
- `orange`
- `brown`
- `red`
- `pink`
- `magenta`
- `violet`

In [ ]:
block_type = ['air', # 0 = unclassified
              'air', # 1 = unassigned
              'dirt', # 2 = ground
              'acacia_bush_leaves', # 3 = low vegetation
              'bush_leaves', # 4 = medium vegetation
              'pine_needles', # 5 = high vegetation
              'tinblock', # 6 = buildings
              'air', # 7 = placeholder
              'air', # 8 = placeholder
              'water_source', # 9 = water
              'air', # 10 = placeholder
              'air', # 11 = placeholder
              'air', # 12 = placeholder
              'air', # 13 = placeholder
              'air'] # 14 = placeholder, add more beyond this if you want

top_layer_of_ground_block = 'dirt_with_grass'
tree_stem_block = 'pine_tree'

## Open Street Maps block types

Many of these options are important for controlling how Open Street Maps is used to render urban worlds. `street_block` is the block type used to paint the streets. `obsidian` is a good choice, offering nice contrast against buildings and vegetation. As well, you have the option to add a slab block type on top of the terrain. Slab blocks are only one-half the height of a normal block, so this can give the impression of a curb raised above the street level. If `add_sidewalk_slab = 1`, then a slab block will be added to the terrain height with `top_layer_of_ground_block` still present underneath the slab.

Note that in order to use any of the `slab` block types, you must use the `csv2terrain` mod that is distributed with this repository.

In [ ]:
street_block = 'obsidian'
add_sidewalk_slab = 1
sidewalk_slab_block = 'slab_stone'
traffic_light_block = 'steelblock'

## Build and interpolate terrain
`get_blocks()` uses `height`, `classification`, and `zyx` to build the `blocks.csv` file that is then used by the [csv2terrain](https://github.com/quantumjim/csv2terrain/blob/master/README.md) Minetest mod to create the map in Minetest. One important role of this function is to fill any "holes" that might exist in the terrain. For example, ground returns may be incorrectly classified or the ground may be obstructed by a surface feature. In both of these cases, there will be a "hole" in the map because there are no known `z` height values at the given `x,y`. Therefore, inverse distance weighting interpolation is applied to the `x,y` locations that need an interpolated `z` value. Through testing, I have found that `knn=5` nearest neighbours is sufficient, but you can also select any value for `k` below depending on the nature of your input LiDAR point cloud and the classified ground returns.

In [ ]:
knn = 5

The function also creates all other classified blocks, including the identification and placement of tree stems and forest canopy. In order to identify the tree stem locations, a local maximum filter is applied to the normalized heights of the canopy (class=5; high vegetation): `canopy elevation - terrain elevation = tree height`. There is a vertical threshold that must be exceeded before a stem is mapped. Since the stem mapping can be sensitive to the type, age, and structure of the forest, the variable `tree_height_percentile` is provided below to control the minimum height of the top of the tree canopy before it may be mapped as a standalone stem. Use `tree_height_percentile = 50` for the median.
**Note: the mapped stems are only for visualization purposes, and may not correspond with the actual location, number or density of true tree stems.**

In [ ]:
tree_height_percentile = 25

In [ ]:
def get_blocks(file,coord,size,scale,knn,block_type,top_layer_of_ground_block,tree_stem_block,tree_height_percentile,street_block,sidewalk_slab_block,add_sidewalk_slab,traffic_light_block):
    height,classification,xyzc = get_points(coord[0],coord[1],file,size,scale)
    
    # check that the number of defined block_types matches the classes in the las file and that ground points are classified
    classes = set(classification.values())
    if 2 not in classes:
        print(datetime.datetime.now(),"    Error: No ground points (class=2) found, but expected. Please check that the las file has ground points classified before proceeding. Exiting.")
        return
    elif len(list(classes-set([2])))==0:
        print(datetime.datetime.now(),"    Warning: No other classes found in the las file. If you were expecting more classes other than ground returns, then check your classification and try again.")
    
    # one layer of dirt for the foundation
    blocks = {}
    for x in range(size):
        for y in range(size):
            blocks[x,y,1] = block_type[2]
    
    # start by first building the terrain from the classified ground points
    ground = {key: value for key, value in classification.items() if value == 2}
    groundHeight = {k:height[k] for k in ground.keys()}
    # matrix to hold the value of the highest ground elevation
    terrainHeight = np.zeros((size, size))
    # subset where class=2, we only want to interpolate the ground surface, not heights of other classes
    xyzc_g = xyzc[xyzc['C']==2]
    for x in range(size):
        if x % 10 == 0:
            print(datetime.datetime.now(),"    Interpolating terrain surface from ground returns line ",x," of ",size)
        for y in range(size):
            # these are the x,y with defined z values
            if (x,y) in groundHeight:
                z = groundHeight[x,y]
                for zz in range(1,z+2):
                    if zz == z+1:
                        # top layer block
                        blocks[x,y,zz] = top_layer_of_ground_block
                        terrainHeight[x,y] = zz
                    else:
                        blocks[x,y,zz] = block_type[2]
            # these are the holes that will be filled with inverse distance weighting interpolation
            else:
                # calculate euclidean distance to all ground points that exist in the point cloud
                ed = np.sqrt((float(x)-xyzc_g['X'].astype(float))**2)+np.sqrt((float(y)-xyzc_g['Y'].astype(float))**2)
                # find the z values of the k nearest neighbours in class 2 only (ground)
                zk = xyzc_g.loc[ed.nsmallest(knn).index,'Z'].astype(float)
                # get the squared euclidean distances (add a small value to ensure no 0 in denominator)
                sd = (ed.nsmallest(knn)**2)+0.0001
                # calculate the inverse distance weighted z value
                z = math.ceil((sum((zk/sd))/sum(1/sd)))
                # fill the hole up to the new z value
                for zz in range(1,z+2):
                    if zz == z+1:
                        # top layer block
                        blocks[x,y,zz] = top_layer_of_ground_block
                        terrainHeight[x,y] = zz
                    else:
                        blocks[x,y,zz] = block_type[2]
    
    # now start building the classified blocks on top of the terrain
    if len(list(classes-set([2])))>0:
        print(datetime.datetime.now(),"    Assembling blocks from classified returns...")
        for cl in classes:
            if cl != 2:
                if cl == 5: 
                    treeHeight = np.zeros((size, size))
                tempclass = {key: value for key, value in classification.items() if value == cl}
                classHeight = {k:height[k] for k in tempclass.keys()}
                for x in range(size):
                    for y in range(size):
                        # these are the x,y with defined z values
                        if (x,y) in classHeight:
                            z = classHeight[x,y]
                            if cl == 5:
                                treeHeight[x,y] = z
                            else:                            
                                for zz in range(int(terrainHeight[x,y]),z):
                                    blocks[x,y,zz] = block_type[cl]
                # now we can derive tree canopy shapes (assumed to be class=5; high vegetation)
                if cl == 5:
                    # standardize tree height to terrain height
                    treeHeightNorm = treeHeight-terrainHeight
                    treeHeightNorm[treeHeightNorm<=0] = 0
                    # we map the stems to the max height x,y position and fill blocks down to the ground
                    # here we use the normalized tree heights to counteract the effect of terrain when mapping the stems
                    # side effect: this will create tree stems "inside" buildings with green roofs or vegetation on roofs
                    maxLocalTreeHeight_xy = peak_local_max(treeHeightNorm, min_distance=5,threshold_abs=np.percentile(treeHeightNorm[treeHeightNorm > 0],q=tree_height_percentile))
                    for (x,y) in classHeight:
                        # get a 3x3 kernel around the target height
                        if x == 0 and y == 0:
                            kern = treeHeight[x:(x+2),y:(y+2)]
                        elif x == size and y == size:
                            kern = treeHeight[(x-1):x,(y-1):y]
                        elif x == 0 and y == size:
                            kern = treeHeight[x:(x+2),(y-1):y]                                
                        elif x == size and y == 0:
                            kern = treeHeight[(x-1):x,y:(y+2)]                                
                        elif x == 0 and y > 0:
                            kern = treeHeight[x:(x+2),(y-1):(y+2)]                                
                        elif x == size and y > 0:
                            kern = treeHeight[(x-1):x,(y-1):(y+2)]                                
                        elif y == 0 and x > 0:
                            kern = treeHeight[(x-1):(x+2),y:(y+2)]
                        elif y == size and x > 0:
                            kern = treeHeight[(x-1):(x+2),(y-1):y]
                        else:
                            kern = treeHeight[(x-1):(x+2),(y-1):(y+2)]
                        kern = kern.astype(int)
                        med = int(np.median(kern[np.nonzero(kern)]))
                        z = int(treeHeight[x,y])
                        # check if the measured height is above or below the local median
                        if z > med:
                            for zz in range(med,z+1):
                                blocks[x,y,zz] = block_type[cl]
                        elif z < med:
                            for zz in range(z,med+1):
                                blocks[x,y,zz] = block_type[cl]
                        elif z == med:
                            # if the height is equivalent to the median then it is likely a single floating voxel
                            # dilate the single floating voxel to make the visual more consistent
                            blocks[x-1,y,z] = block_type[cl]
                            blocks[x+1,y,z] = block_type[cl]
                            blocks[x,y-1,z] = block_type[cl]
                            blocks[x,y+1,z] = block_type[cl]
                            blocks[x,y,z] = block_type[cl]
                            blocks[x,y,z+1] = block_type[cl]
                            blocks[x,y,z-1] = block_type[cl]
                            blocks[x,y,z-2] = block_type[cl]
                    # finally create the blocks for the x,y of the stems
                    for treexy in maxLocalTreeHeight_xy:
                        z = int(treeHeight[treexy[0],treexy[1]])
                        for zz in range(int(terrainHeight[treexy[0],treexy[1]]),z-2):
                            blocks[treexy[0],treexy[1],zz] = tree_stem_block
    
    # Parse the Open Street Maps XML file and look for some map features
    tree = ET.parse('map.osm')
    root = tree.getroot()
    print(datetime.datetime.now(),"    Painting the streets using Open Street Maps data...")
    # these are all the ways (lines that represent buildings or streets)
    for way in root.findall('way'):
        lanes = 0
        flag = 0
        tags = way.findall('tag')
        for tag in tags:
            if tag.get('k')=='lanes':
                lanes = int(tag.get('v'))
            elif tag.get('k')=='name':
                stname = tag.get('v')
        # these are the highways with a lanes attribute (used for width of road in blocks later)
        if lanes > 0:
            print(datetime.datetime.now(),"    Painting "+stname+" "+str(lanes)+" lanes wide")
            # get all the nodes associated with the way that represents the highway
            for nd in way.findall('nd'):
                ndref = nd.get('ref')
                if flag == 0:
                    nodeA = root.findall(".//*[@id='"+ndref+"']")[0]
                    flag = 1
                else:
                    nodeB = root.findall(".//*[@id='"+ndref+"']")[0]
                    alat = float(nodeA.get('lat'))
                    blat = float(nodeB.get('lat'))
                    alon = float(nodeA.get('lon'))
                    blon = float(nodeB.get('lon'))
                    # the current node B will be considered node A when the next node in the way is found
                    nodeA = nodeB
                    # convert latitude and longitude of the node to local UTM
                    autm = p(alon,alat)
                    butm = p(blon,blat)
                    # use some simple linear algebra to calculate the y position of the road given some x in UTM coordinate space
                    abslope = (autm[1]-butm[1])/(autm[0]-butm[0])
                    intercept = autm[1]-abslope*autm[0]
                    # predict the y coordinate from the slope at each x position
                    # xrange here is the range of x between nodes A and B of the way
                    xrange = range(int(autm[0]),int(butm[0]))
                    for xx in xrange:
                        yy = int(xx*abslope+intercept)
                        # rescale to the bounding limits of the map
                        x = int(xx-(coord[0]-(size/2)))
                        y = int(yy-(coord[1]-(size/2)))
                        if x in range(size) and y in range(size):                            
                            # paint the roads based on the lane width
                            # 1 lane = 3 blocks wide
                            # lanes in OSM refers to flowing traffic lanes, but does not include street parking
                            # so for lanes > 1 we add two more lanes to ensure the streets are as wide as they appear in real life
                            if lanes > 1:
                                if (((lanes+2)*3)-1)%2 == 0:
                                    # total road width is even-numbered, so one side of the road will have one more block than the other side
                                    yrange = range(y-int(((lanes+2)*3-1)/2),y+int(((lanes+2)*3)/2))
                                else:
                                    # total road width is odd-numbered, so allocat blocks evenly on either side of the way
                                    yrange = range(y-int(((lanes+2)*3)/2),y+int(((lanes+2)*3)/2))
                                for bb in yrange:
                                        if bb in range(size):
                                            z = terrainHeight[x,bb]
                                            blocks[x,bb,z] = street_block
                            else:
                                # single lane roads are usually alleyways, laneways, etc
                                # so we assume the total width is 3 blocks
                                yrange = range(y-1,y+1)
    # these are individual nodes
    for node in root.findall('node'):
        tags = way.findall('tag')
        # look for traffic signals
        if tag.get('k')=='traffic_signals' or tag.get('v')=='traffic_signals':
            lat = float(node.get('lat'))
            lon = float(node.get('lon'))
            # convert latitude and longitude of the node to local UTM
            utm = p(lon,lat)
            easting = utm[0]
            northing = utm[1]
            # now convert to map coordinates
            x = int(easting-(coord[0]-(size/2)))
            y = int(northing-(coord[1]-(size/2)))
            # traffic signals are 7 meters above the ground
            z = terrainHeight[x,y]
            for zz in range(z,z+7): blocks[x,y,zz] = traffic_light_block
    
    # add sidewalk? ensure you are using the csv2terrain mod distributed with this repository!
    if add_sidewalk_slab == 1: 
        # find all remaining top_layer_of_ground_block blocks
        top_layer = {key: value for key, value in blocks.items() if value == top_layer_of_ground_block}
        for (x,y,z) in top_layer:
            blocks[x,y,z+1] = sidewalk_slab_block
    
    # set the player starting position
    max_height = max(height.values())+2
    x,y = int(size/2),int(size/2)
    if (x,y) in height:
        player = x,height[x,y]+2,y
    else:
        player = x,int(max_height/2),y
        
    # write out the csv file
    print(datetime.datetime.now(),"    Writing out the blocks.csv file now...")
    with open('blocks.csv','w') as file:
        file.write( '0,0,0,min,\n' )
        file.write( str(size)+','+str(max_height)+','+str(size)+','+'max'+',\n' )
        file.write( str(player[0])+','+str(player[1])+','+str(player[2])+','+'player'+',\n' )
        for (x,y,z) in blocks:
            # note that we invert the y-coordinate here with size-y 
            # because this seems to correct the orientation of the world to true cardinal directions
            file.write( str(x)+','+str(z)+','+str(size-y)+','+blocks[x,y,z]+',\n' )

## Run the program

In [ ]:
get_blocks(file,coord,size,scale,knn,block_type,top_layer_of_ground_block,tree_stem_block,tree_height_percentile,street_block,sidewalk_slab_block,add_sidewalk_slab,traffic_light_block)

## Load the map in Minetest
1. Make sure that you have installed the `csv2terrain` mod in Minetest. See instructions above for download and installation.
2. Once the `blocks.csv` file has been generated, copy the csv file to the `\Minetest\mods\csv2terrain\`. Then, run Minetest and create a new world. Ensure that the `Mapgen` mode is set to `singlenode`, which will generate an empty world with only air blocks (called ["nodes"](https://wiki.minetest.net/Games/Minetest_Game/Nodes) in Minetest). You do not need to provide a seed and make sure `Minetest game` is selected under `Game`. Press the `Create` button to generate the world.
3. Once you are redirected back to the main menu, select your world from the `Select World:` menu. For the best experience, I suggest toggling on `Creative Mode` and toggling off `Enable Damage`. Click the `Select Mods` button and here you should see the `csv2terrain` mod if it was installed correctly. Double-click `csv2terrain` and the text will turn green if it has been enabled properly. Press the `Save` button.
4. Click the `Play Game` button. The player should be falling through the air. Press the `/` key to bring up the console. Type `ltbv` (short for: Let There Be Voxels). Be patient as the Minetest world is being generated from the blocks.csv file. If all goes well, you should land on top of the newly built terrain and you can start navigating in the approximate real-life experience using the arrow keys (or `wasd` keys) and `spacebar` to jump.

## Tips and controls
- Press the `/` key and type `grant singleplayer all` into the console to give the player all game privileges.
- Press the `k` key (default) to enable flying, then press the `spacebar` key to fly up and press `shift` to fly down. Press the `k` key again to disable flying.
- Press the `j` key (default) to enable fast mode, which allows you to move/fly quickly around the world and change perspectives. Press the `j` key again to disable fast mode.
- Press the `/` key and type `time 6000` to change the time of day to 6AM causing the Sun to rise. Increments of 1000 are equivalent to hours in 24-hour time (e.g., 18000 is 6PM).
- Press the `I` key to access the inventory of blocks (nodes) in creative mode. Drag blocks into your inventory space at the bottom. `Right-click` on your mouse (default) to place blocks. `Left-click` on your mouse (default) to mine or remove blocks. Press `B` and `N` keys to move left and right, respectively, in your hotbar inventory of blocks.
- Cardinal directions are true in the Minetest world: North in the Minetest world is true Nouth and the Minetest Sun rises in the true East and sets in the true West.